In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
!pwd

/Users/Alenka/code/Tim-Frith/moonster_quakes/notebooks


## Import data for earthquakes

In [3]:
path = "/Users/Alenka/code/Tim-Frith/moonster_quakes/datasets/significant_earthquakes.csv"

In [4]:
earthquakes_df = pd.read_csv(path)

In [5]:
type(earthquakes_df)

pandas.core.frame.DataFrame

In [6]:
earthquakes_df.shape

(97395, 23)

In [7]:
earthquakes_df[earthquakes_df['mag']>7].shape

(1260, 23)

In [8]:
earthquakes_df[earthquakes_df['mag']>7].head()

,Unnamed: 0,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,0,1900-10-09T12:25:00.000Z,57.090,-153.480,NaN,7.86,mw,NaN,NaN,NaN,...,2022-05-09T14:44:17.838Z,"16 km SW of Old Harbor, Alaska",earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,pt
16,16,1904-04-04T10:26:00.880Z,41.758,23.249,15.0,7.02,mw,NaN,NaN,NaN,...,2022-04-26T14:54:31.433Z,"7 km SE of Stara Kresna, Bulgaria",earthquake,NaN,4.8,0.4,NaN,reviewed,iscgem,iscgem
18,18,1904-06-25T21:00:38.720Z,52.763,160.277,30.0,7.70,mw,NaN,NaN,NaN,...,2022-04-25T20:22:48.406Z,"115 km ESE of Petropavlovsk-Kamchatsky, Russia",earthquake,NaN,10.3,0.4,NaN,reviewed,iscgem,iscgem
19,19,1904-06-25T14:45:39.140Z,51.424,161.638,15.0,7.50,mw,NaN,NaN,NaN,...,2022-05-09T22:48:24.972Z,"274 km SE of Petropavlovsk-Kamchatsky, Russia",earthquake,NaN,25.0,0.4,NaN,reviewed,iscgem,iscgem
20,20,1904-08-30T11:43:20.850Z,30.684,100.608,15.0,7.09,mw,NaN,NaN,NaN,...,2022-04-25T20:23:00.657Z,"150 km WNW of Kangding, China",earthquake,NaN,25.0,0.4,NaN,reviewed,iscgem,iscgem


Selected two earthquakes for test calculation in index row 16 and 18

In [9]:
earthquakes_df.loc[16:16]

,Unnamed: 0,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
16,16,1904-04-04T10:26:00.880Z,41.758,23.249,15.0,7.02,mw,NaN,NaN,NaN,...,2022-04-26T14:54:31.433Z,"7 km SE of Stara Kresna, Bulgaria",earthquake,NaN,4.8,0.4,NaN,reviewed,iscgem,iscgem


In [10]:
earthquakes_df.loc[18:18]

,Unnamed: 0,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
18,18,1904-06-25T21:00:38.720Z,52.763,160.277,30.0,7.7,mw,NaN,NaN,NaN,...,2022-04-25T20:22:48.406Z,"115 km ESE of Petropavlovsk-Kamchatsky, Russia",earthquake,NaN,10.3,0.4,NaN,reviewed,iscgem,iscgem


## Test function on 2 earthquakes

In [11]:
earthquakes_df.loc[16:16]['latitude'].values[0]

41.758

In [12]:
first_lat = earthquakes_df.loc[16:16]['latitude'].values[0]
first_lon = earthquakes_df.loc[16:16]['longitude'].values[0]
second_lat = earthquakes_df.loc[18:18]['latitude'].values[0]
second_lon = earthquakes_df.loc[18:18]['longitude'].values[0]

In [13]:
type(first_lat)

numpy.float64

In [14]:
def distances_vectorized(first_lat: float, first_lon: float, second_lat: float, second_lon: float):
    """
    Calculate the haversine and Manhattan distances between two points (specified in decimal degrees).
    Vectorized version for pandas df
    Computes distance in Km
    """
    earth_radius = 6371

    lat_1_rad, lon_1_rad = np.radians(first_lat), np.radians(first_lon)
    lat_2_rad, lon_2_rad = np.radians(second_lat), np.radians(second_lon)

    dlon_rad = lon_2_rad - lon_1_rad
    dlat_rad = lat_2_rad - lat_1_rad

    manhattan_rad = np.abs(dlon_rad) + np.abs(dlat_rad)
    manhattan_km = manhattan_rad * earth_radius

    a = (np.sin(dlat_rad / 2.0)**2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon_rad / 2.0)**2)
    haversine_rad = 2 * np.arcsin(np.sqrt(a))
    haversine_km = haversine_rad * earth_radius

    return dict(
        haversine_in_km = haversine_km,
        manhattan_in_km = manhattan_km
    )

In [15]:
distances_check = distances_vectorized(first_lat, first_lon, second_lat, second_lon)

In [16]:
distances_check

{'haversine_in_km': 8725.08781194689, 'manhattan_in_km': 16460.518575973965}

## Functions from past challenges

Maybe to be used when we are transferring dataframe

In [17]:
# def transform_lonlat_features(X: pd.DataFrame) -> pd.DataFrame:
#     lonlat_features = ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]
#     assert isinstance(X, pd.DataFrame)
#     res = distances_vectorized(X, *lonlat_features)

#     return pd.DataFrame(res)

In [18]:
# def distances_vectorized(df: pd.DataFrame, start_lat: str, start_lon: str, end_lat: str, end_lon: str) -> dict:
#     """
#     Calculate the haversine and Manhattan distances between two points (specified in decimal degrees).
#     Vectorized version for pandas df
#     Computes distance in Km
#     """
#     earth_radius = 6371

#     lat_1_rad, lon_1_rad = np.radians(df[start_lat]), np.radians(df[start_lon])
#     lat_2_rad, lon_2_rad = np.radians(df[end_lat]), np.radians(df[end_lon])

#     dlon_rad = lon_2_rad - lon_1_rad
#     dlat_rad = lat_2_rad - lat_1_rad

#     manhattan_rad = np.abs(dlon_rad) + np.abs(dlat_rad)
#     manhattan_km = manhattan_rad * earth_radius

#     a = (np.sin(dlat_rad / 2.0)**2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon_rad / 2.0)**2)
#     haversine_rad = 2 * np.arcsin(np.sqrt(a))
#     haversine_km = haversine_rad * earth_radius

#     return dict(
#         haversine = haversine_km,
#         manhattan = manhattan_km
#     )